value means srcID, dstID

In [22]:
import dgl
import torch as th
import h5py

In [4]:
# Create a heterograph with 3 node types and 3 edges types.
graph_data = {
    ('drug', 'interacts', 'drug'): (th.tensor([0, 1]), th.tensor([1, 0])),
    ('drug', 'interacts', 'gene'): (th.tensor([0, 1]), th.tensor([2, 3])),
    ('drug', 'treats', 'disease'): (th.tensor([1, 2]), th.tensor([1, 0]))
}
g = dgl.heterograph(graph_data)
g

Graph(num_nodes={'disease': 2, 'drug': 3, 'gene': 4},
      num_edges={('drug', 'interacts', 'drug'): 2, ('drug', 'interacts', 'gene'): 2, ('drug', 'treats', 'disease'): 2},
      metagraph=[('drug', 'drug'), ('drug', 'gene'), ('drug', 'disease')])

In [23]:
f1 = h5py.File('./mesh_path.h5','r')
f2 = h5py.File('./path_reac.h5','r')
f3 = h5py.File('./reac_reac.h5','r')

In [24]:
src1 = f1['src']
src2 = f2['src']
src3 = f3['src']
dis1 = f1['dis']
dis2 = f2['dis']
dis3 = f3['dis']

In [25]:
src2

<HDF5 dataset "src": shape (1289,), type "<i8">

In [26]:
graph_data = {
    ('meshDescriptor', 'hasEvent', 'pathway'): (src1, dis1),
    ('pathway', 'hasEvent', 'reaction'): (src2, dis2),
    ('reaction', 'precedingEvent', 'reaction'): (src3, dis3)
}
g = dgl.heterograph(graph_data)
g

Graph(num_nodes={'meshDescriptor': 121, 'pathway': 209, 'reaction': 1277},
      num_edges={('meshDescriptor', 'hasEvent', 'pathway'): 345, ('pathway', 'hasEvent', 'reaction'): 1289, ('reaction', 'precedingEvent', 'reaction'): 955},
      metagraph=[('meshDescriptor', 'pathway'), ('pathway', 'reaction'), ('reaction', 'reaction')])